# Manual splitting of the prompts

This notebook documents manual splitting steps applied to the prompts table, to be able to replicate manual cleaning steps and to trace back what was changed. Each rule is documented with its purpose and implementation,
The original prompts table remains unchanged - all output goes to `manually_split_prompts`.


In [57]:
import sqlite3
import pandas as pd
import re
from datetime import datetime

conn = sqlite3.connect("../../../data/giicg.db")
prompts= pd.read_sql("SELECT * FROM prompts", conn)
#prompts = prompts.drop_duplicates(subset=['message_text'])
prompts



,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,,Man (cisgender),6
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6
3,1135,55,user,what is FAISS,what is FAISS,,,Man (cisgender),6
4,1137,55,user,Transform given code to process large .mbox file,You are tasked with separating user prompts in...,,Transform given code to process large .mbox file,Man (cisgender),6
...,...,...,...,...,...,...,...,...,...
934,1646,82,user,"def run_query(query, n_results):\n query_em...",this is my code. I want to: Get nodes and edge...,"def run_query(query, n_results):\n query_em...",,Man (cisgender),92
935,1845,37,user,\n nun möchte ich judgement balancing m...,\n nun möchte ich judgement balancing m...,,,Woman (cisgender),29
936,1847,37,user,\n ich sehe keine veränderung im Plot. Was ...,\n ich sehe keine veränderung im Plot. Was ...,,,Woman (cisgender),29
937,1849,2,user,\n I am working on the problem of reconstru...,\n I am working on the problem of reconstru...,,Classic CV - Drone navigation\nIf you ever tho...,Man (cisgender),8


In [58]:
patterns_to_conversational = [
        (r"^some entries are missing:", "message_text", "conversational"),
         (r"^the persona column does not exist anymore and is NOT important!", "message_text", "conversational"),
        (r"^Increase the threshold for orthogogal to 0.1", "message_text", "conversational"),
        (r"^list the attributes of the bmesh class", "message_text", "conversational"),
        (r"^list props of bmedge", "message_text", "conversational"),
        (r"^do this for all linked edges of a bmvertex", "message_text", "conversational"),
        (r"^create a transform matrix for rotation from a 3 orthogonal vectors v1, v2, v3", "message_text", "conversational"),
        (r"^convert it to matrix4x4", "message_text", "conversational"),
        (r"^adjust the above example in the following manner:\s*\n\s*- Do not scale down the image \(it will dull colors\)\s*\n\s*- Randomly sample several thousand pixel in the image\.\s*\n\s*- Covert the pixel information to a format like OKLab\s*\n\s*- Use a clustering algorithm to cluster the images into groups\s*\n\s*- Compute the average color in OKLab space\s*\n\s*- Convert the average color back into RGB\s*\n\s*- print a weighted color palette like in your earlier examples\.", "message_text", "conversational"),
        (r"^the previous version\b", "message_text", "conversational"),
        (r"^der Shape Layer heißt \"Balken\"", "message_text", "conversational"),
        (r"^bitte nur statt -1px -2px einfügen", "message_text", "conversational"),
        (r"rewrite slightly to improve readablity$", "message_text", "conversational"),
        (r"at the top, include check for file encoding and convert to UTF-8 if needed$", "message_text", "conversational"),
        (r"^ohne ace tools", "message_text", "conversational"),
        (r"^Pass `sample_weight` to AutoGluon.how", "message_text", "conversational"),
        (r"the classes are 0 to 9 and are part of the image names\. [a-zA-Z0-9]+-(\d)\.1\.png\s+here \1 is the class", "message_text", "conversational"),
        (r"^this doesnt do anything", "message_text", "conversational"),
        (r"^manipulated_test_files doesnt contain header", "message_text", "conversational"),
        (r"^make it a method", "message_text", "conversational"),
        (r"^rewertie method for knn", "message_text", "conversational"),
        (r"^load it from a file", "message_text", "conversational"),
        (r"^now it doesnt includes entries with only one result", "message_text", "conversational"),
        (r"^when there is only one result is is result. singular", "message_text", "conversational"),
        (r"^i has neither for now", "message_text", "conversational"),
        (r"^nothing happens", "message_text", "conversational"),
        (r"^the last column is also categorical", "message_text", "conversational"),
        (r"^riscrivi stesso contenuto togliendo elenco puntato", "message_text", "conversational"),
        (r"^sistemi un po' grammatica e sintassi", "message_text", "conversational"),
        (r"^correct in the same way", "message_text", "conversational"),
        (r"^Fai lo stesso con:", "message_text", "conversational"),
        (r"^no non va a pagina nuova", "message_text", "conversational"),
        (r"^è ancora a", "message_text", "conversational"),
        (r"e per \\setcounter\{subfigure\}\{\d+\}", "message_text", "conversational"),
        (r"^non funzion", "message_text", "conversational"),
        (r"^non lo prende", "message_text", "conversational"),
        (r"dovrebbe cintennere 5 di questi files.$", "message_text", "conversational"),
        (r"(?<=Uploaded a file)scherzo il file che da problemi è in cluster_data che p questo allegato", "message_text", "conversational"),
        (r"^e le scritte delle stazioni fossero in grassetto e leggermente piu grandi", "message_text", "conversational"),
        (r"Implement the built-in components documentation for svelteflow\.dev\.\s+Orient yourself on the reactflow version\.\s+Also implement all the example code for every component in apps/example-apps/svelte/guides/built-in-components/\.\s+You can take a look how the other 'examples' are implemented for this\.", "message_text", "conversational"),
        (r"^other thing: I have a type in which values in an array can be a number or null. However, the values should never be all number, there needs to be at least one that is undefinded. The array has a lenght of 3.", "message_text", "conversational"),
        (r"^these are the metrics:", "message_text", "conversational"),
        (r"^linux mint", "message_text", "conversational"),
        (r"Folgende Font-Familys: Walter-NeueMager, Walter-NeueMagerKursiv, Walter-NeueMittel, Walter-NeueNormal - bitte als css Einbindung\.\s+Zeilenabstand: 1\.15", "message_text", "conversational"),
        (r"^jetzt im Body Schrift \"Walter-NeueMager\"", "message_text", "conversational"),
        (r"css: designe <title> mit Font \"Walter-NeueMittel\"", "message_text", "conversational"),
        (r"^align des audio-elements: links (oder width 100%)", "message_text", "conversational"),
        (r"^float left, width 100%, margin-left 0 funktionieren alle nicht", "message_text", "conversational"),
        (r"^audio controls sitzt in einer <figure>", "message_text", "conversational"),
        (r"^nich tunterstrichen", "message_text", "conversational"),
        (r"^the header should have the same characteristics as the \"content\" class -- in terms of indentation to the left with different medias in use.", "message_text", "conversational"),
        (r"^set font color", "message_text", "conversational"),
        (r"^Transform given code to process large .mbox file", "message_text", "conversational"),
        (r"^Here as well please:", "message_text", "conversational"),
        (r"i want to factor out this part of my code to a new file called dataloader.py. help me do that", "message_text", "conversational"),
        (r"rewrite this so that it doenst include the print statement$", "message_text", "conversational"),
        (r"^currently it looks like this", "message_text", "conversational"),
        (r"heres how the cligen data looks like\.\s*\n\s*\n\s*Adjust my code to handle cligen data laike in the code i previously sent\. tell me wehre the changes happened", "message_text", "conversational"),
        (r"like this\?$", "message_text", "conversational"), #
        (r"please print the file name in a copyable way after each plot$", "message_text", "conversational"),
        (r"^this is how manipulated_test_files looks like:", "message_text", "conversational"),
        (r"this is how manipulated_test_files looks like:|this is how the other looks_", "message_text", "conversational"),
        (r"please remove these features from this method:", "message_text", "conversational"),
        (r"please turn to method$", "message_text", "conversational"),
        (r"it should be created here$", "message_text", "conversational"),
        (r"So this is how the entrys look in imported functions$", "message_text", "conversational"),
        (r"ignore the file name when clustering by only using numerical columns$", "message_text", "conversational"),
        (r"this is how the output should be formatted:", "message_text", "conversational"),
        (r"^please use bash", "message_text", "conversational"),
        (r"^please dont change the formatting", "message_text", "conversational"),
        (r"^please only show relevant part", "message_text", "conversational"),
        (r"riscrivi lo stesso rimuovendo elenco$", "message_text", "conversational"),
        (r"^inside a <figure>\?", "message_text", "conversational"), #
        (r"^insert a line break after each first line of the paragraph", "message_text", "conversational"),
        (r"^from the title and subtitle, please remove the h1 and h2 categories", "message_text", "conversational"),
        (r"^and you can unload source code pro", "message_text", "conversational"),
    ]

In [59]:
patterns_from_conversational = [
    (r"febfb3e1396c96d38cd0f9b1",  "conversational", "other"),
    (r"single=fff8e8f7eed3aab396674636", "conversational", "other"),
    (r'\[\{"likes":"1266","tags":"beige sage brown vintage nature fall food warm earth","date":"1 week"\}\]', "conversational", "other"),
    (r"the header Content-Type is application/x-www-form-urlencoded; charset=UTF-8\s+the endpoint is https://colorhunt\.co/php/feed\.php\s+the request body is \{'step': 0, 'sort': 'random', 'tags': ''\}", "conversational", "other"),
    (r'"lastName.trim() is not a function"$', "conversational", "other"),
    (r'"Error 21: undefined is not an object. Line: 44$', "conversational", "other"),
    (r'"Error 1302: No such element Line: 68 ->          var playButtonLayer = group.artLayers.getByName("Play Button");"$', "conversational", "other"),
    (r'"Error 21: undefined is not an object. Line: 10$', "conversational", "other"),
    (r'" 3\.4\.3 Early-fusion\n\nFor early-fusion, two different approaches are tested: i) the concatenation of all modality features; and ii) the combination of the features through a mean operator. Since the original feature vectors of the various modalities have different shapes, we start by projecting them into the same dimension using a set of MLPs. Then we proceed with the early fusion. In the scenario where one or more modalities are missing and we do not perform feature reconstruction, we either replace the corresponding features by a vector of zeros (concatenation) or exclude the modality (masked mean). Finally, the resulting ouput is fed to the classifier."', "conversational", "other"),
    (r"Models\s+Base CT\s+Base MRI\s+Base Path\s+Base Clingen\s+Weighted Sum\s+Learn Weights\s+Mean\s+Concat\s*\nEpochs\s+60\s+60\s+60\s+60\s+60\s+60\s+120\s+120\s*\nArchitecture\s+3L\s+3L\s+3L\s+3L\s+3L\s+3L\s+5L\s+5L\s*\nHidden Sizes\s+128\s+128\s+128\s+128\s+128\s+128\s+128\s+128\s*\nInitial LR\s+1e-3\s+1e-3\s+1e-4\s+1e-3\s+1e-3\s+1e-3\s+1e-3\s+1e-3\s*\nLR Scheduler\s+Cosine\s+Cosine\s+Const\s+Const\s+Const\s+Cosine\s+Cosine\s+Cosine\s*\nLR Settings\s+Default\s+Default\s+None\s+None\s+None\s+Default\s+Default\s+Default\s*\nOptimizer\s+AdamW\s+SGD\s+SGD\s+AdamW\s+AdamW\s+AdamW\s+AdamW\s+SGD\s*\nOversample\s+None\s+None\s+6x\s+6x\s+6x\s+6x\s+6x\s+6x\s*\nBatch Size\s+14\s+14\s+14\s+14\s+14\s+14\s+14\s+14\s*\nBatch Norm\s+No\s+No\s+No\s+No\s+Yes\s+Yes\s+Yes\s+Yes", "conversational", "other"),
    (r"Experiment\s+CT\s+MRI\s+WSI\s+ClinGen\s+All Patients\s*\n\nEarly Fusion Cat\s+77\.50\s+85\.00\s+78\.45\s+78\.45\s+78\.45", "conversational", "other"),
    (r"Data Preprocessing & Normalization:\nSubtle differences in how features are scaled or normalized across modalities can affect convergence\. The study might have applied a more refined normalization or imputation strategy that leads to better feature fusion\.", "conversational", "other"),
    (r"'intrusion detection'': About 249 results \(0,11 sec\)\n2024-06-23 16:38:59,294 - INFO - Completed 183 searches, 183 successful\.\n  6%\|▌         \| 183/3068 \[25:44<6:49:23,  8\.51s/it\]2024-06-23 16:39:07,809 - INFO - Results text for query '\"Acquire Access\" \+ \"Botnet\" \+ anomaly \+ 'intrusion detection'': About 85 results \(0,09 sec\)\n2024-06-23 16:39:07,809 - INFO - Completed 184 searches, 184 successful\.\n  6%\|▌         \| 184/3068 \[25:53<6:49:13,  8\.51s/it\]2024-06-23 16:39:16,319 - INFO - Results text for query '\"Acquire Access\" \+ \"Web Services\" \+ anomaly \+ 'intrusion detection'': About 63 results \(0,10 sec\)\n2024-06-23 16:39:16,319 - INFO - Completed 185 searches, 185 successful\.\n  6%\|▌         \| 185/3068 \[26:01<6:49:03,  8\.51s/it\]2024-06-23 16:39:24,765 - INFO - Results text for query '\"Acquire Access\" \+ \"Serverless\" \+ anomaly \+ 'intrusion detection'': 4 results \(0,08 sec\)", "conversational", "other"),
    (r"Checking for NaN values in the dataset\.\.\.\nNo NaN values detected\.\nTesting min_samples=5, xi=0\.05, min_cluster_size=0\.01, n_components=20\.\.\.\nError with parameters min_samples=5, xi=0\.05, min_cluster_size=0\.01, n_components=20: Input contains NaN\.", "conversational", "other"),
    (r"    # check with even number of points\n    x1 = jnp\.geomspace\(1\.0, 10\.0, 100\)", "conversational", "other"),
    (r"100 0\.3333333333333333 0\.3483835051532227", "conversational", "other"),
    (r"nils@raspberrypi:~/Documents \$ python starter\.py wave 60\nTraceback \(most recent call last\):\n  File \"/home/nils/Documents/starter\.py\", line 4, in <module>\n    from updatemode import update_mode\n  File \"/home/nils/Documents/updatemode\.py\", line 3, in <module>\n    from starter import start_process\n  File \"/home/nils/Documents/starter\.py\", line 4, in <module>\n    from updatemode import update_mode\nImportError: cannot import name 'update_mode' from partially initialized module 'updatemode' \(most likely due to a circular import\) \(/home/nils/Documents/updatemode\.py\)\nnils@raspberrypi:~/Documents \$", "conversational", "other"),
    (r'if __name__ == "__main__": ', "conversational", "other"),
    (r"Cannot update a component (ModalLayout) while rendering a different component (RouteComponent). To locate the bad setState() call inside RouteComponent, follow the stack trace as described when tryping to call openModal from within a Layout Component", "conversational", "other"),
    (r"AttributeError: 'Abalone' object has no attribute 'plot_expanded'", "conversational", "other"),

]

In [60]:
def create_literal_pattern(text):
    """Convert literal text to exact regex pattern"""
    return re.escape(text)


patterns_from_conversational = [
    (create_literal_pattern("febfb3e1396c96d38cd0f9b1"), "conversational", "other"),
    (create_literal_pattern("single=fff8e8f7eed3aab396674636"), "conversational", "other"),
    (create_literal_pattern(
        '[{"likes":"1266","tags":"beige sage brown vintage nature fall food warm earth","date":"1 week"}]'),
     "conversational", "other"),
    (r"\s*the header Content-Type is application/x-www-form-urlencoded; charset=UTF-8\s*\n\s*the endpoint is https://colorhunt\.co/php/feed\.php\s*\n\s*the request body is \{'step': 0, 'sort': 'random', 'tags': ''\}\s*",
     "conversational", "other"),
    (create_literal_pattern('"lastName.trim() is not a function"'), "conversational", "other"),
    (r"\"Error 21: undefined is not an object\. Line: 44", "conversational", "other"),
    (create_literal_pattern(
        '"Error 1302: No such element Line: 68 ->          var playButtonLayer = group.artLayers.getByName("Play Button");"'),
     "conversational", "other"),
    (r"\"Error 21: undefined is not an object\. Line: 10\n->      for \(var i = 0; i < pathItems\.length; i\+\+\) \{\"", "conversational", "other"),
    (create_literal_pattern(
        '" 3.4.3 Early-fusion\n\nFor early-fusion, two different approaches are tested: i) the concatenation of all modality features; and ii) the combination of the features through a mean operator. Since the original feature vectors of the various modalities have different shapes, we start by projecting them into the same dimension using a set of MLPs. Then we proceed with the early fusion. In the scenario where one or more modalities are missing and we do not perform feature reconstruction, we either replace the corresponding features by a vector of zeros (concatenation) or exclude the modality (masked mean). Finally, the resulting ouput is fed to the classifier."'),
     "conversational", "other"),
    (create_literal_pattern(
        "Models\nBase CT\nBase MRI\nBase Path\nBase Clingen\nWeighted Sum\nLearn Weights\nMean\nConcat\nEpochs\n60\n60\n60\n60\n60\n60\n120\n120\nArchitecture\n3L\n3L\n3L\n3L\n3L\n3L\n5L\n5L\nHidden Sizes\n128\n128\n128\n128\n128\n128\n128\n128\nInitial LR\n1e-3\n1e-3\n1e-4\n1e-3\n1e-3\n1e-3\n1e-3\n1e-3\nLR Scheduler\nCosine\nCosine\nConst\nConst\nConst\nCosine\nCosine\nCosine\nLR Settings\nDefault\nDefault\nNone\nNone\nNone\nDefault\nDefault\nDefault\nOptimizer\nAdamW\nSGD\nSGD\nAdamW\nAdamW\nAdamW\nAdamW\nSGD\nOversample\nNone\nNone\n6x\n6x\n6x\n6x\n6x\n6x\nBatch Size\n14\n14\n14\n14\n14\n14\n14\n14\nBatch Norm\nNo\nNo\nNo\nNo\nYes\nYes\nYes\nYes"),
     "conversational", "other"),
    (create_literal_pattern(
        "Experiment\nCT\nMRI\nWSI\nClinGen\nAll Patients\n\nEarly Fusion Cat\n77.50\n85.00\n78.45\n78.45\n78.45"),
     "conversational", "other"),
    (create_literal_pattern(
        "Data Preprocessing & Normalization:\nSubtle differences in how features are scaled or normalized across modalities can affect convergence. The study might have applied a more refined normalization or imputation strategy that leads to better feature fusion."),
     "conversational", "other"),
    (create_literal_pattern(
        "'intrusion detection'': About 249 results (0,11 sec)\n2024-06-23 16:38:59,294 - INFO - Completed 183 searches, 183 successful.\n  6%|▌         | 183/3068 [25:44<6:49:23,  8.51s/it]2024-06-23 16:39:07,809 - INFO - Results text for query '\"Acquire Access\" + \"Botnet\" + anomaly + 'intrusion detection'': About 85 results (0,09 sec)\n2024-06-23 16:39:07,809 - INFO - Completed 184 searches, 184 successful.\n  6%|▌         | 184/3068 [25:53<6:49:13,  8.51s/it]2024-06-23 16:39:16,319 - INFO - Results text for query '\"Acquire Access\" + \"Web Services\" + anomaly + 'intrusion detection'': About 63 results (0,10 sec)\n2024-06-23 16:39:16,319 - INFO - Completed 185 searches, 185 successful.\n  6%|▌         | 185/3068 [26:01<6:49:03,  8.51s/it]2024-06-23 16:39:24,765 - INFO - Results text for query '\"Acquire Access\" + \"Serverless\" + anomaly + 'intrusion detection'': 4 results (0,08 sec)"),
     "conversational", "other"),
    (create_literal_pattern(
        "Checking for NaN values in the dataset...\nNo NaN values detected.\nTesting min_samples=5, xi=0.05, min_cluster_size=0.01, n_components=20...\nError with parameters min_samples=5, xi=0.05, min_cluster_size=0.01, n_components=20: Input contains NaN."),
     "conversational", "other"),
    (create_literal_pattern("    # check with even number of points\n    x1 = jnp.geomspace(1.0, 10.0, 100)"),
     "conversational", "other"), #
    (create_literal_pattern("100 0.3333333333333333 0.3483835051532227"), "conversational", "other"), #
    (create_literal_pattern(
        "nils@raspberrypi:~/Documents $ python starter.py wave 60\nTraceback (most recent call last):\n  File \"/home/nils/Documents/starter.py\", line 4, in <module>\n    from updatemode import update_mode\n  File \"/home/nils/Documents/updatemode.py\", line 3, in <module>\n    from starter import start_process\n  File \"/home/nils/Documents/starter.py\", line 4, in <module>\n    from updatemode import update_mode\nImportError: cannot import name 'update_mode' from partially initialized module 'updatemode' (most likely due to a circular import) (/home/nils/Documents/updatemode.py)\nnils@raspberrypi:~/Documents $"),
     "conversational", "other"), #
    (create_literal_pattern('if __name__ == "__main__": '), "conversational", "other"), #
    (create_literal_pattern(
        "Cannot update a component (ModalLayout) while rendering a different component (RouteComponent). To locate the bad setState() call inside RouteComponent, follow the stack trace as described when tryping to call openModal from within a Layout Component"),
     "conversational", "other"), #
    (r"AttributeError: 'Abalone' object has no attribute 'plot_expanded'$", "conversational",
     "other"),
    (r"record_id\s+record_timestamp\s+updated_timestamp\s+report_dt\s+source\s+metric_name\s+metric_num\s+metric_value", "conversational", "other"),
    (r"report_dt\s+source\s+metric_name\s+metric_num\s+metric_value", "conversational", "other"),#
]


In [61]:
# Move to conversational
def move_misplaced_content(df, patterns):
    """Move prompts from source to target column"""
    moved_count = 0
    for pattern, source_column, target_column in patterns:
        mask = df[source_column].str.contains(pattern, regex=True, case=False, na=False)
        if mask.any():
            for idx in df[mask].index:
                cell_content = df.loc[idx, source_column]
                if pd.notna(cell_content):
                    # Find the match
                    match = re.search(pattern, cell_content, re.IGNORECASE | re.MULTILINE)
                    if match:
                        matched_text = match.group(0)
                        # Move only the matched text to conversational
                        df.loc[idx, target_column] = matched_text

                        # If source column is 'conversational', remove the matched content
                        if source_column == 'conversational':
                            remaining_text = cell_content.replace(matched_text, '', 1).strip()
                            df.loc[idx, source_column] = remaining_text if remaining_text else None

                        moved_count += 1


    print(f"Moved content for {moved_count} rows from other to conversational")
    return df

In [62]:
def apply_all_rules(df):
    """Apply all cleaning rules in sequence"""
    print(f"Starting with {len(df)} rows")
    print("-" * 40)

    df = move_misplaced_content(df, patterns_to_conversational)
    df = move_misplaced_content(df, patterns_from_conversational)

    print("-" * 40)
    print(f"Final: {len(df)} rows")
    return df

# Apply rules
cleaned_prompts = apply_all_rules(prompts.copy())



Starting with 939 rows
----------------------------------------
Moved content for 76 rows from other to conversational
Moved content for 22 rows from other to conversational
----------------------------------------
Final: 939 rows


/var/folders/w6/f5trsq6n2jx87f1mqgd_4vv80000gn/T/ipykernel_68735/1066254049.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[source_column].str.contains(pattern, regex=True, case=False, na=False)
/var/folders/w6/f5trsq6n2jx87f1mqgd_4vv80000gn/T/ipykernel_68735/1066254049.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[source_column].str.contains(pattern, regex=True, case=False, na=False)


In [63]:
differences = prompts.compare(cleaned_prompts)
# Get rows that have any differences
altered_rows_mask = ~prompts.eq(cleaned_prompts).all(axis=1)
altered_df = cleaned_prompts[altered_rows_mask]

altered_df


,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6
4,1137,55,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,Transform given code to process large .mbox file,Man (cisgender),6
57,748,34,user,Here as well please:\nimport numpy as np\n\nfr...,Here as well please:,import numpy as np\n\nfrom .ecg_tools import E...,Here as well please:\n ---\...,Woman (cisgender),16
67,55,8,user,the persona column does not exist anymore and ...,the persona column does not exist anymore and ...,,the persona column does not exist anymore and ...,Woman (cisgender),28
106,95,12,user,Increase the threshold for orthogogal to 0.1,Increase the threshold for orthogogal to 0.1,,Increase the threshold for orthogogal to 0.1,Man (cisgender),34
...,...,...,...,...,...,...,...,...,...
880,1033,53,user,insert a line break after each first line of t...,insert a line break after each first line of t...,insert a line break after each first line of t...,"Some prompts may only contain code, some only ...",Man (cisgender),91
892,1057,53,user,"from the title and subtitle, please remove the...","from the title and subtitle, please remove the...","from the title and subtitle, please remove the...","Some prompts may only contain code, some only ...",Man (cisgender),91
896,1065,53,user,the header should have the same characteristic...,the header should have the same characteristic...,,the header should have the same characteristic...,Man (cisgender),91
899,1071,53,user,set font color,set font color,,set font color,Man (cisgender),91


In [64]:
cleaned_prompts.to_sql('manually_split_prompts', conn, if_exists='replace', index=False)

939